# Import Required Libraries
Import libraries such as pandas, numpy, and joblib for model loading and evaluation.

In [3]:
# Import Required Libraries
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# Model Accuracy Evaluation for Longitude and Latitude Prediction
This notebook loads both the original and best-tuned XGBoost models, predicts on the test set, and calculates an accuracy score based on how close the predicted values are to the true values.

## Load Data and Prepare Test Set

In [4]:
df = pd.read_csv('../clean_data/Master5G.csv')
X = df[['hour', 'speed']]
y_long = df['longitude']
y_lat = df['latitude']
X_train, X_test, y_long_train, y_long_test = train_test_split(X, y_long, test_size=0.2, random_state=42)
_, _, y_lat_train, y_lat_test = train_test_split(X, y_lat, test_size=0.2, random_state=42)

/tmp/ipykernel_35529/858522556.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../clean_data/Master5G.csv')


## Load Both Sets of Models

In [5]:
# Original models
xgb_long = XGBRegressor()
xgb_lat = XGBRegressor()
xgb_long.load_model('../position_model/xgb_longitude_model.pt')
xgb_lat.load_model('../position_model/xgb_latitude_model.pt')

# Best-tuned models
xgb_long_best = XGBRegressor()
xgb_lat_best = XGBRegressor()
xgb_long_best.load_model('../position_model/xgb_longitude_model_best.pt')
xgb_lat_best.load_model('../position_model/xgb_latitude_model_best.pt')

## Predict and Calculate Accuracy Scores

In [6]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers. Use 3956 for miles
    R = 6371.0

    # Convert coordinates from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    # Haversine formula
    a = math.sin(delta_phi / 2)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c  # in kilometers
    return distance


In [15]:
# Predict
y_long_pred = xgb_long.predict(X_test)
y_lat_pred = xgb_lat.predict(X_test)
y_long_pred_best = xgb_long_best.predict(X_test)
y_lat_pred_best = xgb_lat_best.predict(X_test)

# Calculate accuracy within 50 meters (0.05 km) using haversine distance
def accuracy_within_1km(y_lat_true, y_long_true, y_lat_pred, y_long_pred):
    correct = 0
    total = len(y_lat_true)
    for lat_t, lon_t, lat_p, lon_p in zip(y_lat_true, y_long_true, y_lat_pred, y_long_pred):
        dist = haversine(lat_t, lon_t, lat_p, lon_p)
        if dist <= 1:
            correct += 1
    return correct / total if total > 0 else 0

# Calculate accuracy for original models
acc_orig_1km = accuracy_within_1km(y_lat_test.values, y_long_test.values, y_lat_pred, y_long_pred)
# Calculate accuracy for best-tuned models
acc_best_1km = accuracy_within_1km(y_lat_test.values, y_long_test.values, y_lat_pred_best, y_long_pred_best)

print(f'Original Model Accuracy within 1km: {acc_orig_1km:.3f}')
print(f'Best-tuned Model Accuracy within 1km: {acc_best_1km:.3f}')

Original Model Accuracy within 1km: 0.057
Best-tuned Model Accuracy within 1km: 0.057
